In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,0,Bluff,NZ,-46.6000,168.3333,47.80,88,100,10.71,2022-10-26 17:44:00
1,1,Tiksi,RU,71.6872,128.8694,0.63,99,68,6.40,2022-10-26 17:44:42
2,2,Bengkulu,ID,-3.8004,102.2655,74.41,91,100,4.41,2022-10-26 17:44:43
3,3,Atuona,PF,-9.8000,-139.0333,77.13,72,15,18.68,2022-10-26 17:44:43
4,4,Taoudenni,ML,22.6783,-3.9836,88.90,10,57,9.22,2022-10-26 17:44:44


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? ")) 

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [7]:
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
2,2,Bengkulu,ID,-3.8004,102.2655,74.41,91,100,4.41,2022-10-26 17:44:43
3,3,Atuona,PF,-9.8000,-139.0333,77.13,72,15,18.68,2022-10-26 17:44:43
7,7,Hirara,JP,24.8000,125.2833,77.11,67,93,17.69,2022-10-26 17:44:50
16,16,Labuhan,ID,-6.8844,112.2051,76.60,88,100,12.68,2022-10-26 17:45:11
17,17,Bambous Virieux,MU,-20.3428,57.7575,73.65,78,75,11.50,2022-10-26 17:45:11
20,20,Arraial Do Cabo,BR,-22.9661,-42.0278,76.96,65,40,19.57,2022-10-26 17:45:13
23,23,Chuy,UY,-33.6971,-53.4616,70.21,84,100,13.40,2022-10-26 17:45:15
26,26,Rikitea,PF,-23.1203,-134.9692,73.22,84,90,15.37,2022-10-26 17:45:30
34,34,Puerto Ayora,EC,-0.7393,-90.3518,73.36,91,93,8.99,2022-10-26 17:45:42
35,35,Bethel,US,41.3712,-73.4140,73.49,71,100,3.44,2022-10-26 17:45:43


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID       185
City          185
Country       184
Lat           185
Lng           185
Max Temp      185
Humidity      185
Cloudiness    185
Wind Speed    185
Date          185
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesnÃ¢Â€Â™t have empty rows.

clean_df = preferred_cities_df

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Bengkulu,ID,74.41,-3.8004,102.2655,
3,Atuona,PF,77.13,-9.8000,-139.0333,
7,Hirara,JP,77.11,24.8000,125.2833,
16,Labuhan,ID,76.60,-6.8844,112.2051,
17,Bambous Virieux,MU,73.65,-20.3428,57.7575,
20,Arraial Do Cabo,BR,76.96,-22.9661,-42.0278,
23,Chuy,UY,70.21,-33.6971,-53.4616,
26,Rikitea,PF,73.22,-23.1203,-134.9692,
34,Puerto Ayora,EC,73.36,-0.7393,-90.3518,
35,Bethel,US,73.49,41.3712,-73.4140,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [16]:
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
# 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
# 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
 # 6e. Make request and retrieve the JSON data from the search.
    try:
        hotels = requests.get(base_url, params=params).json()   
    
# 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("City not found. Skipping...")
        pass

City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...


In [17]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Bengkulu,ID,74.41,-3.8004,102.2655,Grage Hotel Bengkulu
3,Atuona,PF,77.13,-9.8000,-139.0333,Villa Enata
7,Hirara,JP,77.11,24.8000,125.2833,Hotel Atoll Emerald Miyakojima
16,Labuhan,ID,76.60,-6.8844,112.2051,DoubleFAG
17,Bambous Virieux,MU,73.65,-20.3428,57.7575,Casa Tia Villa
...,...,...,...,...,...,...
668,Chibombo,ZM,78.76,-14.6569,28.0706,Chibombo District Council Guest House
671,Ban Nahin,LA,71.44,18.2418,104.2122,
673,Mayumba,GA,77.29,-3.4320,10.6554,LIKWALE LODGE
675,Kendari,ID,75.56,-3.9450,122.4989,Hotel Wixel


In [18]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']

In [26]:
clean_hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Bengkulu,ID,74.41,-3.8004,102.2655,Grage Hotel Bengkulu
3,Atuona,PF,77.13,-9.8000,-139.0333,Villa Enata
7,Hirara,JP,77.11,24.8000,125.2833,Hotel Atoll Emerald Miyakojima
16,Labuhan,ID,76.60,-6.8844,112.2051,DoubleFAG
17,Bambous Virieux,MU,73.65,-20.3428,57.7575,Casa Tia Villa
...,...,...,...,...,...,...
667,Chino,US,73.60,34.0122,-117.6889,"Motel 6 Chino, CA - Los Angeles Area"
668,Chibombo,ZM,78.76,-14.6569,28.0706,Chibombo District Council Guest House
673,Mayumba,GA,77.29,-3.4320,10.6554,LIKWALE LODGE
675,Kendari,ID,75.56,-3.9450,122.4989,Hotel Wixel


In [27]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

In [28]:
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} Ã‚Â°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))